In [97]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
import jax
import jax.numpy as jnp
from jax import random

import optax
from numpyro.infer import SVI, Trace_ELBO, TraceEnum_ELBO

In [99]:
from src.models.M2VAE import M2VAE

from src.models.encoder_decoder import MNISTEncoder, MNISTDecoder
from src.data_loading.constants import MNIST_IMG_SHAPE

from src.data_loading.loaders import get_data_loaders

In [100]:
m2_vae = M2VAE(MNISTEncoder, MNISTDecoder, 10, 30, MNIST_IMG_SHAPE)

In [101]:
optimizer = optax.adam(1e-3)

In [102]:
svi = SVI(m2_vae.model, 
            m2_vae.guide, 
            optim=optimizer, 
            loss=TraceEnum_ELBO(max_plate_nesting=1)
)

In [103]:
state = svi.init(random.PRNGKey(0), xs=jnp.ones((1,)+MNIST_IMG_SHAPE), ys=jnp.ones((1), dtype=jnp.int32))

In [104]:
svi.get_params(state).keys()

dict_keys(['decoder$params', 'encoder1$params', 'encoder2$params'])

In [105]:
seed = 42

# Data loading
img_shape, loader_dict = get_data_loaders(dataset_name="MNIST", 
                                          p_test=0.2, 
                                          p_val=0.1, 
                                          p_supervised=0.2, 
                                          batch_size=2, 
                                          num_workers=0, 
                                          seed=seed)

Successfully loaded MNIST dataset.
Total num samples 60000
Num test samples: 12000
Num validation samples: 4800
Num supervised samples: 8640
Num unsupervised samples: 34560


In [106]:
tot_loss_supervised = 0.0
tot_loss_unsupervised = 0.0

c = 0
for is_supervised, batch in loader_dict["semi_supervised"]:
    if is_supervised:
        x, y = batch
        state, loss = svi.update(state, xs=x, ys=y)
        tot_loss_supervised += loss
    else:
        x = batch
        state, loss = svi.update(state, xs=x)
        tot_loss_unsupervised += loss
    if c > 5:
        break
    c+= 1
print(tot_loss_supervised, tot_loss_unsupervised)

c:\Users\Matthieu Dinot\Desktop\MVA\PGM\venv\Lib\site-packages\jax\_src\linear_util.py:191: UserWarning: Missing a plate statement for batch dimension -1 at site 'z'. You can use `numpyro.util.format_shapes` utility to check shapes at all sites of your model.
  ans = self.f(*args, **dict(self.params, **kwargs))
c:\Users\Matthieu Dinot\Desktop\MVA\PGM\venv\Lib\site-packages\jax\_src\linear_util.py:191: UserWarning: Missing a plate statement for batch dimension -2 at site 'z'. You can use `numpyro.util.format_shapes` utility to check shapes at all sites of your model.
  ans = self.f(*args, **dict(self.params, **kwargs))
c:\Users\Matthieu Dinot\Desktop\MVA\PGM\venv\Lib\site-packages\jax\_src\linear_util.py:191: UserWarning: Missing a plate statement for batch dimension -1 at site 'y'. You can use `numpyro.util.format_shapes` utility to check shapes at all sites of your model.
  ans = self.f(*args, **dict(self.params, **kwargs))
c:\Users\Matthieu Dinot\Desktop\MVA\PGM\venv\Lib\site-package

10861.868 0.0
